# Introduction to ESG Bulk - Python

ESG stands for Environmental, Social and (Corporate) Governance data.

Refinitiv Data Platform (RDP) provides simple web based API access to a broad range of content, including ESG content and ESG content in bulk.

With growing popularity of socially conscious investing, Refinitiv offers one of the most comprehensive Environmental, Social and Governance (ESG) databases in the industry, covering over 80% of global market cap, across more than 450 different ESG metrics, with history going back to 2002. Customers looking to download our ESG content can do so through our bulk API service in Refinitiv Data Platform (RDP). RDP is a cloud based API that provides a single access point to all Refinitiv content.

ESG data is the first content made available in our bulk API service known as Client File Store (CFS). This capability allows our customers to download our entire history of ESG coverage. To more about how the ESG Bulk Service works in Refinitiv Data Platform, please visit:

https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs

Within RDP family of service, ESG Bulk is part of Client File Store (CFS) - based section of service, find out more at:

https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis

Let us now focus on the programmatic interaction with ESG Bulk RDP service.

In [1]:
import requests, json, time, getopt, sys
import pandas as pd

### Set Valid Credentials 

Valid RDP credentials are required to proceed:
* USERNAME
* PASSWORD
* CLIENTID

To read one's valid credentials from a file (that can be shared by many code examples), leave below code as is.

To provide credentials in place:
* uncomment
* replace the commented credentials with one's valid assigned credentials
* comment the read from file step readCredsFromFile

In [9]:
USERNAME = "VALIDUSER"
PASSWORD = "VALIDPASSWORD"
CLIENT_ID = "SELFGENERATEDCLIENTID"

def readCredsFromFile(filePathName):
### Read valid credentials from file
    global USERNAME, PASSWORD, CLIENT_ID
    credFile = open(filePathName,"r")    # one per line
                                                #--- RDP MACHINE ID---
                                                #--- LONG PASSWORD---
                                                #--- GENERATED CLIENT ID---

    USERNAME = credFile.readline().rstrip('\n')
    PASSWORD = credFile.readline().rstrip('\n')
    CLIENT_ID = credFile.readline().rstrip('\n')

    credFile.close()

readCredsFromFile("..\creds\credFileHuman.txt")

# Make sure that creds are read in correctly
#print("USERNAME="+str(USERNAME))
#print("PASSWORD="+str(PASSWORD))
#print("CLIENT_ID="+str(CLIENT_ID))

### Set Application Constants

In [10]:
# Set Application Constants
RDP_AUTH_VERSION = "/v1"
RDP_ESG_BULK_VERSION = "/v1"
RDP_BASE_URL = "https://api.refinitiv.com"
RDP_ESG_BUCKET = "ESG"
CATEGORY_URL = "/auth/oauth2"
ENDPOINT_URL = "/token"
CLIENT_SECRET = ""
TOKEN_FILE = "token.txt"
SCOPE = "trapi"

### Define Token Handling and Obtain a Valid Token

Having a valid token is a pre-requisite to requesting of any RDP content, and will be passed into the next steps.

In [11]:
TOKEN_ENDPOINT = RDP_BASE_URL + CATEGORY_URL + RDP_AUTH_VERSION + ENDPOINT_URL

def _requestNewToken(refreshToken):
    if refreshToken is None:
        tData = {
            "username": USERNAME,
            "password": PASSWORD,
            "grant_type": "password",
            "scope": SCOPE,
            "takeExclusiveSignOnControl": "true"
        };
    else:
        tData = {
            "refresh_token": refreshToken,
            "grant_type": "refresh_token",
        };

    # Make a REST call to get latest access token
    response = requests.post(
        TOKEN_ENDPOINT,
        headers = {
            "Accept": "application/json"
        },
        data = tData,
        auth = (
            CLIENT_ID,
            CLIENT_SECRET
        )
    )
    
    if response.status_code != 200:
        raise Exception("Failed to get access token {0} - {1}".format(response.status_code, response.text));

    # Return the new token
    return json.loads(response.text);

def saveToken(tknObject):
    tf = open(TOKEN_FILE, "w+");
    print("Saving the new token");
    # Append the expiry time to token
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10;
    # Store it in the file
    json.dump(tknObject, tf, indent=4)
    
def getToken():
    try:
        print("Reading the token from: " + TOKEN_FILE);
        # Read the token from a file
        tf = open(TOKEN_FILE, "r+")
        tknObject = json.load(tf);

        # Is access token valid
        if tknObject["expiry_tm"] > time.time():
            # return access token
            return tknObject["access_token"];

        print("Token expired, refreshing a new one...");
        tf.close();
        # Get a new token from refresh token
        tknObject = _requestNewToken(tknObject["refresh_token"]);

    except Exception as exp:
        print("Caught exception: " + str(exp))
        print("Getting a new token using Password Grant...");
        tknObject = _requestNewToken(None);

    # Persist this token for future queries
    saveToken(tknObject)
    print("Token is: " + tknObject["access_token"])
    # Return access token
    return tknObject["access_token"];

accessToken = getToken();
print("Have token now");

Reading the token from: token.txt
Token expired, refreshing a new one...
Caught exception: Failed to get access token 400 - {"error":"invalid_grant"   } 
Getting a new token using Password Grant...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwicFJOemxqT3NsRUhMcFZ4WnZJaUtwMTJFd1NTSDZVLVhjWDNNdHpXTTR2LU93ZzBXQUFsM2dzNGpaZVI3SDJ5S0NKbThGVk9wNHhsRHZWVGlQY0xyWFJmQkNYQm5MWC1EVUNDTjc3ajBkYzVXZFd4VlhRbi13bVlEdVRNcmpiZ1VMOGxWcWFNcnU4RmZPbTE1ZDJ1SmprQWRwZW1MVDdMVTlYV1NCZjNNTTJZT3hiQVhtWXAyWThFV1hTWTI2MzlEMWZ2WVhLN0NwSkZoWVEtRWd1TktEMUlKVXptcjBsbmJmeEpvcjJISUEzak9SS19iVDE3MVJFVnhQY0pBeGlucUFUY1liN3JCX0Y1bVpzMkE4dUh3d283RDB5ZjNnXzZWS0FhYl9JdUZ5VUNINzRjS0xsNEpxV28xQWc3bE50Zy1Eb3dWLUxBSmRIMEw4NmgzVGhnZ0JCVlMtS3EtQjBGM0ZKbDJYbGdfNVBDbEVXR0N3VUNmaFBuNUNDNXNnbWlUNlJkRjRicGI2T0JwSjRmYUdyZUVjM2E3Q3c3VVVRLVA3NHd6cHR1UXZkRUREWnRVdE9MemYxenZzeDc2OGRQNGpZUzJERTJRdHZDaEFaZHRCOHRh

## Request Available File Sets

The purpose of ESG bulk service is obtaining ESG content in bulk.  The content is available as:

•	A full JSON data file containing history for all measures and all organizations.
•	A delta JSON data file that contains only incremental changes to the universe since last week. 

A customer can examine the available File Sets and is expected to:

•	Build the initial ESG representation with the full files
•	Apply delta, changes, as they become available
•	Fill a gap if the retrival was not completed and the content missed remains available

This step serves to verify the type of the file, for example:
•	ESGRawFullScheme
•	ESGScoresFull
•	ESGScoresWealthFull

In [29]:
FILESET_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/file-sets?bucket='+ RDP_ESG_BUCKET
FILESET_ID = ''
PACKAGE_ID = ''
WHICH_FILESET_ID_TO_STORE = 1   # 10 are returned per page
WHICH_PACKAGE_ID_TO_STORE = 2   

def requestFileSets(token, withNext, skipToken, attributes):   
    global FILESET_ENDPOINT
    global FILESET_ID
    global PACKAGE_ID
    global WHICH_FILESET_ID_TO_STORE   
    
    print("Obtaining FileSets in ESG Bucket...")
  
    FILESET_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/file-sets?bucket='+ RDP_ESG_BUCKET
    
    querystring = {}
    payload = ""
    jsonfull = ""
    jsonpartial = ""
    
    headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + token,
            'cache-control': "no-cache"
    }

    if attributes:
        FILESET_ENDPOINT = FILESET_ENDPOINT + attributes
    if withNext:
        FILESET_ENDPOINT = FILESET_ENDPOINT + '&skipToken=' +skipToken
        
    response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
         
    print('Raw response=');
    print(response);
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        print('Parsed json response=');
        print(json.dumps(jsonFullResp, indent=2));
        # We are going to store FileSet ID of the second File Set retrieved for future reference
        FILESET_ID = jsonFullResp['value'][WHICH_FILESET_ID_TO_STORE]['id']
        print('FILESET_ID stored is: '+str(FILESET_ID))
        PACKAGE_ID = jsonFullResp['value'][WHICH_PACKAGE_ID_TO_STORE]['packageId']
        print('PACKAGE_ID stored is: '+str(PACKAGE_ID))
        return jsonFullResp; 
    else:
        return '';

jsonFullResp = requestFileSets(accessToken, False, '','');

Obtaining FileSets in ESG Bucket...
Reading the token from: token.txt
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4022-f0c0-e3968404-9146-1b496d4e11b4",
      "name": "RFT-ESG-Sources-Current-Delta-2021-01-10",
      "bucketName": "ESG",
      "packageId": "4867-9a46-216e838a-9241-8fc3561b51ef",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Sources"
        },
        {
          "name": "ResultCount",
          "value": "6888"
        }
      ],
      "files": [
        "4f94-ff55-90b727d0-b896-3ecd59dd8855"
      ],
      "numFiles": 1,
      "availableFrom": "2021-01-10T14:38:04Z",
      "availableTo": "2021-02-10T14:38:03Z",
      "status": "READY",
      "created": "2021-01-10T14:38:04Z",
      "modified": "2021-01-10T14:38:06Z"
    },
    {
      "id": "403f-4d1f-9a6bec16-bc83-cdcabba066b0",
      "name": "RFT-ESG-Sources-Full-Init-2021-01-17",
      "bucketName": "ESG",
      "packageId": "4308

### Present Raw Results in Tabular View

In [30]:
import pandas as pd
df = pd.json_normalize(jsonFullResp['value'])
df

,id,name,bucketName,packageId,attributes,files,numFiles,availableFrom,availableTo,status,created,modified,contentFrom,contentTo
0,4022-f0c0-e3968404-9146-1b496d4e11b4,RFT-ESG-Sources-Current-Delta-2021-01-10,ESG,4867-9a46-216e838a-9241-8fc3561b51ef,"[{'name': 'ContentType', 'value': 'ESG Sources...",[4f94-ff55-90b727d0-b896-3ecd59dd8855],1,2021-01-10T14:38:04Z,2021-02-10T14:38:03Z,READY,2021-01-10T14:38:04Z,2021-01-10T14:38:06Z,NaN,NaN
1,403f-4d1f-9a6bec16-bc83-cdcabba066b0,RFT-ESG-Sources-Full-Init-2021-01-17,ESG,4308-bc80-2054dc20-83db-6224911311d0,"[{'name': 'ResultCount', 'value': '3655472'}, ...","[410f-1fd2-f0275b88-a4ae-599711569f7f, 4156-2a...",17,2021-01-18T09:29:50Z,2021-02-01T09:29:49Z,READY,2021-01-18T09:29:50Z,2021-01-18T09:31:42Z,NaN,NaN
2,4053-80f9-53e95bc3-8d9f-5df48df990ad,RFT-ESG-Sources-Current-Init-2021-01-24,ESG,4867-9a46-216e838a-9241-8fc3561b51ef,"[{'name': 'ResultCount', 'value': '3661884'}, ...","[4151-d940-34a4bd67-963a-83f5be5e7fea, 4180-65...",18,2021-01-24T16:07:34Z,2021-02-24T16:07:34Z,READY,2021-01-24T16:07:34Z,2021-01-24T16:09:28Z,1970-01-01T00:00:00Z,2021-01-24T14:30:00Z
3,407e-5d23-1e37e696-840d-5e9a34f87540,RFT-ESG-Scores-Wealth-Standard-init-2020-12-27,ESG,4bcc-4602-0a57ebb2-baf0-1fc9825e76b6,"[{'name': 'ContentType', 'value': 'ESG Scores'}]",[41fb-c32c-d6d2f164-892c-fba87f28a0cd],1,2020-12-27T16:19:10Z,2021-01-27T16:19:09Z,READY,2020-12-27T16:19:10Z,2020-12-27T16:19:13Z,NaN,NaN
4,40ad-6a3a-da6a7e51-b267-f0ee50973176,RFT-ESG-Raw-Current-SchemeA-init-2021-01-10,ESG,4c62-b05c-2a529a9d-81b3-224eacd50379,"[{'name': 'ContentType', 'value': 'ESG Raw Cur...",[499b-48d1-6b36b902-96a1-7fe9f241328e],1,2021-01-10T16:24:54Z,2021-02-10T16:24:53Z,READY,2021-01-10T16:24:54Z,2021-01-10T16:25:10Z,NaN,NaN
5,40b2-c0dc-5190083e-a488-9203bd8b0fd1,RFT-ESG-Raw-Current-SchemeB-Env-Delta-2020-12-27,ESG,4cbb-e27e-318835e3-bad7-dee7a0ebc3b0,"[{'name': 'ContentType', 'value': 'ESG Raw Cur...",[4c18-5889-8c01101f-b619-97f1c2e230c4],1,2020-12-27T16:31:34Z,2021-01-27T16:31:34Z,READY,2020-12-27T16:31:34Z,2020-12-27T16:31:41Z,NaN,NaN
6,40db-4f5f-d840e2a8-afc6-33ec41fd434c,RFT-ESG-Scores-Full-Init-2021-01-17,ESG,42de-14b7-37470ec8-9087-ccd1a1bae75d,"[{'name': 'ResultCount', 'value': '73681'}, {'...",[4379-b70a-db66fefe-b92e-71e9dde76abc],1,2021-01-18T08:31:09Z,2021-02-01T08:31:09Z,READY,2021-01-18T08:31:09Z,2021-01-18T08:31:15Z,NaN,NaN
7,4110-80a2-586214e2-adec-0ef0f59b8382,RFT-ESG-Raw-Full-SchemeA-Init-2021-01-17,ESG,4e6a-ca79-af368ff5-931d-d2781b1cdb85,"[{'name': 'ContentType', 'value': 'ESG Raw Ful...","[4607-eda1-997b59e6-bfca-7aa600ad9277, 4667-04...",6,2021-01-17T12:50:34Z,2021-01-31T12:50:34Z,READY,2021-01-17T12:50:34Z,2021-01-17T12:51:26Z,NaN,NaN
8,4125-f4ac-2a33f80c-a165-ca4355ddfcd9,RFT-ESG-Raw-Current-SchemeB-Env-Init-2021-01-10,ESG,4cbb-e27e-318835e3-bad7-dee7a0ebc3b0,"[{'name': 'ContentType', 'value': 'ESG Raw Cur...",[4909-fc14-65b8374a-b735-4ed171dd5609],1,2021-01-10T16:32:59Z,2021-02-10T16:32:59Z,READY,2021-01-10T16:32:59Z,2021-01-10T16:33:07Z,NaN,NaN
9,4126-fa19-763b8cb5-8ee9-97d59cec68b2,RFT-ESG-Scores-Full-Init-2021-01-24,ESG,42de-14b7-37470ec8-9087-ccd1a1bae75d,"[{'name': 'ContentType', 'value': 'ESG Scores'...",[42a0-9ec5-c0994aa3-b2cf-5b1666f56cbd],1,2021-01-24T13:37:14Z,2021-02-07T13:37:13Z,READY,2021-01-24T13:37:14Z,2021-01-24T13:37:19Z,1970-01-01T00:00:00Z,2021-01-24T13:25:00Z


### Paginate Through the Available FileSets
(interrupt at any point)

In [4]:
i = 1
while "@nextLink" in jsonFullResp: 
    print('<<< Iteraction: '+str(i)+' >>>  More exists: '+ jsonFullResp['@nextLink'] + ', skipToken is: ' + jsonFullResp['@nextLink'][-62:]+'\n')
    jsonFullResp = requestFileSets(accessToken, True, jsonFullResp['@nextLink'][-62:],'');
    i+=1;
print('Last response without next=');
print(jsonFullResp);

<<< Iteraction: 1 >>>  More exists: /file-store/v1/file-sets?bucket=ESG&skipToken=ZmlsZXNldElkPTQ0YjUtYTJiNy0wZDExNDAxMC1hM2Y1LTEzMmIzMWZkNDViZA, skipToken is: ZmlsZXNldElkPTQ0YjUtYTJiNy0wZDExNDAxMC1hM2Y1LTEzMmIzMWZkNDViZA

Obtaining FileSets in ESG Bucket...
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "44e0-490a-2a5471e1-8ce9-dfecd4666d0d",
      "name": "RFT-ESG-Sources-Full-Delta-2021-01-03",
      "bucketName": "ESG",
      "packageId": "4308-bc80-2054dc20-83db-6224911311d0",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Sources"
        },
        {
          "name": "ResultCount",
          "value": "5337"
        }
      ],
      "files": [
        "4caa-2c19-988fba80-bc20-275ee390efd4"
      ],
      "numFiles": 1,
      "availableFrom": "2021-01-03T14:12:51Z",
      "availableTo": "2021-01-17T14:12:50Z",
      "status": "READY",
      "created": "2021-01-03T14:12:51Z",
      "modified": "2021-

### Retrieve FileSets of Specific File Type (Filter By Attribute and By PackageId)
The file types may change over time, at the time of this writing, the available FileSets are of types:

* ESG Raw Full A
* ESG Raw Full B
* ESG Raw Current A
* ESG Raw Current B
* ESG Sources
* ESG Raw Wealth Standard

* Symbology Cusip
* Symbology SEDOL
* Symbology Organization
* Symbology Instrument Quote

Further, the selected package, if also filtering by packageId has to contain the files per filtering arrtibutes, in order to request their listing succefully,
otherwise the result will be empty.


In [25]:
requestFileSets(accessToken, False, '','&attributes=ContentType:Symbology Cusip');

Obtaining FileSets in ESG Bucket...
Reading the token from: token.txt
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "45fa-8626-3e24243b-a482-5bd455ff8f27",
      "name": "RFT-ESG-Symbology-Cusip-Init-2021-01-21",
      "bucketName": "ESG",
      "packageId": "466c-c470-22a4cab3-8d0c-770cdfa5fc45",
      "attributes": [
        {
          "name": "ContentType",
          "value": "Symbology Cusip"
        }
      ],
      "files": [
        "4570-69ac-2faaede3-9879-fb1e190147fc"
      ],
      "numFiles": 1,
      "contentFrom": "1970-01-01T00:00:00Z",
      "contentTo": "2021-01-21T10:20:00Z",
      "availableFrom": "2021-01-21T10:27:10Z",
      "availableTo": "2021-02-04T10:27:10Z",
      "status": "READY",
      "created": "2021-01-21T10:27:10Z",
      "modified": "2021-01-21T10:27:13Z"
    },
    {
      "id": "4cec-1cc0-809bd4ce-834c-13372d224719",
      "name": "RFT-ESG-Symbology-Cusip-Init-2021-01-14",
      "bucketName": "ESG",
      "pac

IndexError: list index out of range

In [26]:
requestFileSets(accessToken, False, '','&packageId='+PACKAGE_ID+'&attributes=ContentType:ESG Sources');

Obtaining FileSets in ESG Bucket...
Reading the token from: token.txt
Raw response=
<Response [200]>
Parsed json response=
{
  "value": [
    {
      "id": "4022-f0c0-e3968404-9146-1b496d4e11b4",
      "name": "RFT-ESG-Sources-Current-Delta-2021-01-10",
      "bucketName": "ESG",
      "packageId": "4867-9a46-216e838a-9241-8fc3561b51ef",
      "attributes": [
        {
          "name": "ContentType",
          "value": "ESG Sources"
        },
        {
          "name": "ResultCount",
          "value": "6888"
        }
      ],
      "files": [
        "4f94-ff55-90b727d0-b896-3ecd59dd8855"
      ],
      "numFiles": 1,
      "availableFrom": "2021-01-10T14:38:04Z",
      "availableTo": "2021-02-10T14:38:03Z",
      "status": "READY",
      "created": "2021-01-10T14:38:04Z",
      "modified": "2021-01-10T14:38:06Z"
    },
    {
      "id": "4053-80f9-53e95bc3-8d9f-5df48df990ad",
      "name": "RFT-ESG-Sources-Current-Init-2021-01-24",
      "bucketName": "ESG",
      "packageId": "4

### Retrieve Complete File Details of FileSet ID

In a previous step we have stored a FileSet ID that we are about to use for the demonstartion of this feature.

In [27]:
FILES_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files?filesetId='+ FILESET_ID
WHICH_FILE_ID_TO_STORE = 0
FILE_ID = ''
FILE_NAME = ''
 
def requestFileDetails(token):   
    global FILES_ENDPOINT
    global FILE_ID
    global FILE_NAME
    print("Obtaining File details..." )
  
    querystring = {}
    payload = ""
    jsonfull = ""
    jsonpartial = ""
    
    headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + token,
            'cache-control': "no-cache"
    }
        
    response = requests.request("GET", FILES_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILES_ENDPOINT, data=payload, headers=headers, params=querystring)
         
    print('Raw response=');
    print(response);
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        print('Parsed json response=');
        print(json.dumps(jsonFullResp, indent=2));
        FILE_ID = jsonFullResp['value'][WHICH_FILE_ID_TO_STORE]['id']
        print('FILE_ID stored is: '+str(FILE_ID))
        FILE_NAME = jsonFullResp['value'][WHICH_FILE_ID_TO_STORE]['filename']
        print('FILE_NAME stored is: '+str(FILE_NAME))
        return jsonFullResp; 
    else:
        return '';

jsonFullResp = requestFileDetails(accessToken);

Obtaining File details...
Reading the token from: token.txt
Token expired, refreshing a new one...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwiZjh4ZFhVTmhMajl6aVRxRnhyMkFRdE00Y3VtN2c5MktvVkhvNXdzeEt2TElFd0tJVExBVWdpalFRVnA1dnByUXhaNDhsWnQ0RUhzSlBJNWZGb2U1eDdCajVUVThxOElXbnR3VU5JUDB3ZVoxdTZoMDhIMG5ZZjhmcDFSdnY0WWZpejEweTJrOHc2SF9fUmM2LXhFMEt2MTZEemYzWkljbFFSRDNOcWowM3Z0YkRYeV8xanpLVUQyelk2S3RBak1nMi1jampSaWw2ZHU4WUZORHBuaFRYWC0wUmd3bTEzdVdMb0xiYTBDeXhxQVV2NWNMV2JldUhnOTR4OUpBWjVmNVFUTUFIMERXUlZQSDhJaXMwbVFCRnRHd3JPZmdVNTg2aHowZXhieTlSQkVIeGtHSmRIZUFZbDFEdVN2VEJsY3NUc1RnWnVNQTBWYkk1dGZOekpaUzlFS3ByN1hiVmRrUFl0WU5UNUg5NUZvNnBPVVlsUmNaT21qMmpYSWtKbGZQTjdfWEphLWs1cmt1YS11V3I0b0cxRGFwWUh5bXg4STM1YkM5bHR6QlpSOFJYeGFka2dMQzl0R2lYaU5Fc18tbHI5MDY0U0FyLVU2Z0JGaTNQMC1WMml1dGRMMlFaYVRkc0tHb3RicC1kaXZyb0NrUnZZZFlWelp4a2ozdVdhQXdxbThIVDU1dDVFUTh2OVh1R2RZbTRhWGV0Z2JyYkV

###  Stream File via File Id using Redirect

In [6]:
import shutil

FILES_STREAM_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files/'+ FILE_ID+ '/stream'
 
def requestFileDownload(token):   
    global FILES_STREAM_ENDPOINT
    print("Obtaining File ... " + FILES_STREAM_ENDPOINT)
  
    filename = FILE_NAME
    chunk_size = 1000
    
    headers = {
            'Authorization': 'Bearer ' + token,
            'cache-control': "no-cache",
            'Accept': '*/*'
    }
        
    response = requests.request("GET", FILES_STREAM_ENDPOINT, headers=headers, stream=True, allow_redirects=True)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET",FILES_STREAM_ENDPOINT, headers=headers, stream=True, allow_redirects=True)

         
    print('Response code=' + str(response.status_code));
    
    if response.status_code == 200:
        print('Processing...')
        with open(filename, 'wb') as fd:
            shutil.copyfileobj(response.raw, fd) 
        print('Look for gzipped file named: '+ filename + ' in current directory')
        response.connection.close()
        
    return; 


requestFileDownload(accessToken);

Obtaining File ... https://api.refinitiv.com/file-store/v1/files/4400-9f13-7f9f2f37-8e8d-f8d26840be99/stream
Reading the token from: token.txt
Token expired, refreshing a new one...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwib09hc2J0aWY0OXZkVC1jZDVsZVJXYUc4dzZWaTYxVjN4TXNBWWlOWV9rd3pCdGplMWpKbEZrUW9VQkg3Zmx0blZQeVMwTFU2RVN6ZVU5UVplLU1YbFFaSm1kYlNVTmRXcmQ4d0MzNnFIdFpxSFVDVkxwbGpkaGZ6UzE2UE1yYTFVNk1FNzVEZVp6Um5iLXNCQWxsdllyZktBY3N4WmNWQjh1ZExjWXFXVVRPMXNaSndwalk5b0NHNnNxT1dXdWVqdWk1ZzZacVgweTc4TG05cjV1cmkyODl5SDhnRjBzVVhvSV94M2tmS2RZMnd0Z25OdjRSOExacl9oMTQzUS1TOGNyMGx2X0hQRWhmS0w3Ukc2RklPcmZWNzEtdkRkNkdpdXBWd3FwRkUySDhKMGJfQjYwd01VMHBfOVN6Y0lXN2UwQ2dJZC01c3lNcXlXZ1djVHByX1d0aU9uUk9BajhiQlYxQmFqUldveFY2Y3U3VHB1QUM1aGh4aTdrVnJ2OWZmSGRybUxhQVVwaTdqX0hhVWt3aGhrbXBhbElpcjNmcnZmU1BlQ3VMMW9WbVhhSVpGaVRqMlk5dm1uOUxSNmNqM2pGYVVidWZOZE5WcXpJTjVobXVkN3hqTmtpQWU1aGU3

### Get File Location (Step 1 of 2)

In [17]:
import shutil

FILES_STREAM_ENDPOINT = RDP_BASE_URL+'/file-store'+RDP_ESG_BULK_VERSION + '/files/'+ FILE_ID+ '/stream?doNotRedirect=true'
DIRECT_URL = ''
 
def requestFileLocation(token):   
    
    global FILES_STREAM_ENDPOINT
    global DIRECT_URL
    
    print("Obtaining File ... " + FILES_STREAM_ENDPOINT)
  
    filename = FILE_NAME
    chunk_size = 1000
    
    headers = {
            'Authorization': 'Bearer ' + token,
            'cache-control': "no-cache",
            'Accept': '*/*'
    }
        
    response = requests.request("GET", FILES_STREAM_ENDPOINT, headers=headers, stream=False, allow_redirects=False)
    
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET",FILES_STREAM_ENDPOINT, headers=headers, stream=False, allow_redirects=False)

         
    print('Response code=' + str(response.status_code));
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        print('Parsed json response=');
        print(json.dumps(jsonFullResp, indent=2));
        DIRECT_URL = jsonFullResp['url'];
        print('File Direct URL is: '  +str(DIRECT_URL)+ '|||');
        
    return; 


requestFileLocation(accessToken);

Obtaining File ... https://api.refinitiv.com/file-store/v1/files/47cb-2400-e6f524d4-8703-c3941bf8ab58/stream?doNotRedirect=true
Reading the token from: token.txt
Response code=200
Parsed json response=
{
  "url": "https://a206464-prod-esg.s3.amazonaws.com/ESG_Symbology/2020/12/24/RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz?x-request-Id=536d09e8-091e-4186-b7c7-b7a3e06eba2a&x-package-id=466c-c470-22a4cab3-8d0c-770cdfa5fc45&x-client-app-id=GE-A-01103867-3-603&x-file-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz&x-fileset-id=458d-60a7-cfe476de-9d35-117f124921aa&x-bucket-name=ESG&x-uuid=GENTC-25929&x-file-Id=47cb-2400-e6f524d4-8703-c3941bf8ab58&x-fileset-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGsaCXVzLWVhc3QtMSJIMEYCIQD%2Ftq0Gm8hMY8rR0Ag4aFsCbP2NGl5Po5rupc5VofvGdAIhAMrnxq8I48iN0QP8mNAOdXj1LNflv2yrWJeCMU1AZHBqKskDCDQQAhoMNjQyMTU3MTgxMzI2IgxC%2B7h4k42jyuQfpFAqpgNcYR9DULH7s5EOnrAJY69xSemjO3toR

### Download File From File Location (Step 2 of 2)

In [20]:
from urllib.parse import urlparse, parse_qs
def requestDirectFileDownload(token):   
    
    global DIRECT_URL
    print("Obtaining File from URL... " + DIRECT_URL)
    
    #Parse out URL parameters for submission into requests
    url_obj = urlparse(DIRECT_URL)
    parsed_params = parse_qs(url_obj.query)
    # extract the URL without query parameters
    parsed_url = url_obj._replace(query=None).geturl()

    response = requests.get(parsed_url, params=parsed_params,stream=True)
        
    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.get(parsed_url, params=query)

         
    print('Response code=' + str(response.status_code));        
  
    filename = 'another_'+FILE_NAME    
    
    if response.status_code == 200:
        print('Processing...')
        with open(filename, 'wb') as fd:
            shutil.copyfileobj(response.raw, fd) 

        print('Look for gzipped file named: '+ filename + ' in current directory')
        response.connection.close()
        
    return; 


requestDirectFileDownload(accessToken);

Obtaining File from URL... https://a206464-prod-esg.s3.amazonaws.com/ESG_Symbology/2020/12/24/RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz?x-request-Id=536d09e8-091e-4186-b7c7-b7a3e06eba2a&x-package-id=466c-c470-22a4cab3-8d0c-770cdfa5fc45&x-client-app-id=GE-A-01103867-3-603&x-file-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24.jsonl.gz&x-fileset-id=458d-60a7-cfe476de-9d35-117f124921aa&x-bucket-name=ESG&x-uuid=GENTC-25929&x-file-Id=47cb-2400-e6f524d4-8703-c3941bf8ab58&x-fileset-name=RFT-ESG-Symbology-Cusip-Init-2020-12-24&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGsaCXVzLWVhc3QtMSJIMEYCIQD%2Ftq0Gm8hMY8rR0Ag4aFsCbP2NGl5Po5rupc5VofvGdAIhAMrnxq8I48iN0QP8mNAOdXj1LNflv2yrWJeCMU1AZHBqKskDCDQQAhoMNjQyMTU3MTgxMzI2IgxC%2B7h4k42jyuQfpFAqpgNcYR9DULH7s5EOnrAJY69xSemjO3toReCmEPStxmr41vVCDV8%2Foacwygzh3LQunFgInznvLQQumzPXRupZXRbBuaIwLQBE29Il5XRHITBSnAd8mfBOE0jOrM3HaUXFwwGe%2Bb5Jz3g24ej4uFtgffYjhLLZek0rZjqc%2BZmBPK7jJ7bnoni%2BD9H6FVnAvc8WTRLEo5HmdLtfGK8IFLr%